In [88]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

df= pd.read_csv('/kaggle/input/advertising/advertising.csv')
df= df.drop('City', axis= 1)
df['Timestamp']= pd.to_datetime(df['Timestamp'])
# Any results you write to the current directory are saved as output.

/kaggle/input/advertising/advertising.csv


In [89]:
print(df.shape)
df.head()

(1000, 9)


,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,Male,Country,Timestamp,Clicked on Ad
0,68.95,35,61833.90,256.09,Cloned 5thgeneration orchestration,0,Tunisia,2016-03-27 00:53:11,0
1,80.23,31,68441.85,193.77,Monitored national standardization,1,Nauru,2016-04-04 01:39:02,0
2,69.47,26,59785.94,236.50,Organic bottom-line service-desk,0,San Marino,2016-03-13 20:35:42,0
3,74.15,29,54806.18,245.89,Triple-buffered reciprocal time-frame,1,Italy,2016-01-10 02:31:19,0
4,68.37,35,73889.99,225.58,Robust logistical utilization,0,Iceland,2016-06-03 03:36:18,0


In [90]:
px.bar(df, x= 'Age', y= 'Daily Time Spent on Site', color= 'Clicked on Ad',
       title= 'What age group is more likely to click on ads')

**Analysis without time and percent_usage**

In [91]:
# Analysis without time and percent_usage
df1= df.drop(['Ad Topic Line','Timestamp', 'Country'], axis=1)
df1.head()

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Male,Clicked on Ad
0,68.95,35,61833.90,256.09,0,0
1,80.23,31,68441.85,193.77,1,0
2,69.47,26,59785.94,236.50,0,0
3,74.15,29,54806.18,245.89,1,0
4,68.37,35,73889.99,225.58,0,0


In [92]:
X= df1.drop('Clicked on Ad', axis= 1)
y= df1['Clicked on Ad']

# split with equal number of 1s and 0s in 'Clicked on Ad' column
for i in range(0,42):
    X_train, X_test, y_train, y_test = tts(X, y, train_size= 0.7, random_state= i)
    if(y_train.value_counts()[0] == y_train.value_counts()[1]):
        print('Random state= ', i)
        break
        
print(y_train.value_counts())
print(y_test.value_counts())

Random state=  4
1    350
0    350
Name: Clicked on Ad, dtype: int64
1    150
0    150
Name: Clicked on Ad, dtype: int64


In [93]:
data={'Model': [], 'Accuracy':[], 'Prediction': [], 'Recall':[], 'F1 score': [], 'Time taken':[]}
conclusion= pd.DataFrame(data)
conclusion

,Model,Accuracy,Prediction,Recall,F1 score,Time taken


Logistic Regression

In [94]:
# this one
import time
start= time.time()

logr= LogisticRegression(penalty='l1', random_state= 42)  # lasso regularization
logr.fit(X_train, y_train)
y_pred= logr.predict(X_test)

print(confusion_matrix(y_test, y_pred))
conclusion=conclusion.append({'Model': 'LogisticRegression', 'Accuracy':accuracy_score(y_test, y_pred),
                  'Prediction': precision_score(y_test, y_pred),
                  'Recall':recall_score(y_test, y_pred, average='weighted'),
                  'F1 score': f1_score(y_test, y_pred, average='weighted'),
                  'Time taken':time.time()- start},
                 ignore_index=True)

conclusion

[[148   2]
 [  8 142]]


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.



,Model,Accuracy,Prediction,Recall,F1 score,Time taken
0,LogisticRegression,0.966667,0.986111,0.966667,0.966653,0.021988


Naive Bayes

In [95]:
import time
start= time.time()
gnb= GaussianNB()
gnb.fit(X_train, y_train)
y_pred_gnb= gnb.predict(X_test)

print(confusion_matrix(y_test, y_pred_gnb))
conclusion=conclusion.append({'Model': 'Naive Bayes', 'Accuracy':accuracy_score(y_test, y_pred_gnb),
                  'Prediction': precision_score(y_test, y_pred_gnb),
                  'Recall':recall_score(y_test, y_pred_gnb, average='weighted'),
                  'F1 score': f1_score(y_test, y_pred_gnb, average='weighted'),
                  'Time taken':time.time()- start},
                 ignore_index=True)

conclusion

[[144   6]
 [  6 144]]


,Model,Accuracy,Prediction,Recall,F1 score,Time taken
0,LogisticRegression,0.966667,0.986111,0.966667,0.966653,0.021988
1,Naive Bayes,0.960000,0.960000,0.960000,0.960000,0.018969


K Nearest Neighbour

In [96]:
import time
start= time.time()
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X, y)
y_pred_n= neigh.predict(X_test)

print(confusion_matrix(y_test, y_pred_n))
conclusion= conclusion.append({'Model': 'K nearest neighbour', 'Accuracy':accuracy_score(y_test, y_pred_n),
                  'Prediction': precision_score(y_test, y_pred_n),
                  'Recall':recall_score(y_test, y_pred_n, average='weighted'),
                  'F1 score': f1_score(y_test, y_pred_n, average='weighted'),
                  'Time taken':time.time()- start},
                 ignore_index=True)

conclusion

[[137  13]
 [ 22 128]]


,Model,Accuracy,Prediction,Recall,F1 score,Time taken
0,LogisticRegression,0.966667,0.986111,0.966667,0.966653,0.021988
1,Naive Bayes,0.960000,0.960000,0.960000,0.960000,0.018969
2,K nearest neighbour,0.883333,0.907801,0.883333,0.883228,0.030479


Decision Tree

In [97]:
import time
start= time.time()

dtc= DecisionTreeClassifier(random_state=0, splitter='random')
dtc.fit(X_train, y_train)
y_pred_dtc= dtc.predict(X_test)

print(confusion_matrix(y_test, y_pred_dtc))
conclusion= conclusion.append({'Model': 'DecisionTreeClassifier', 'Accuracy':accuracy_score(y_test, y_pred_dtc),
                  'Prediction': precision_score(y_test, y_pred_dtc),
                  'Recall':recall_score(y_test, y_pred_dtc, average='weighted'),
                  'F1 score': f1_score(y_test, y_pred_dtc, average='weighted'),
                  'Time taken':time.time()- start},
                 ignore_index=True)

conclusion

[[140  10]
 [ 12 138]]


,Model,Accuracy,Prediction,Recall,F1 score,Time taken
0,LogisticRegression,0.966667,0.986111,0.966667,0.966653,0.021988
1,Naive Bayes,0.960000,0.960000,0.960000,0.960000,0.018969
2,K nearest neighbour,0.883333,0.907801,0.883333,0.883228,0.030479
3,DecisionTreeClassifier,0.926667,0.932432,0.926667,0.926663,0.014130


Random Forest

In [117]:
import time
start= time.time()

rfc= RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
rfc.fit(X, y)
y_pred_rfc= rfc.predict(X_test)

print(confusion_matrix(y_test, y_pred_rfc))
conclusion= conclusion.append({'Model': 'RandomForestClassifier', 'Accuracy':accuracy_score(y_test, y_pred_rfc),
                  'Prediction': precision_score(y_test, y_pred_rfc),
                  'Recall':recall_score(y_test, y_pred_rfc, average='weighted'),
                  'F1 score': f1_score(y_test, y_pred_rfc, average='weighted'),
                  'Time taken':time.time()- start},
                 ignore_index=True)

conclusion

[[144   6]
 [  7 143]]


,Model,Accuracy,Prediction,Recall,F1 score,Time taken
0,LogisticRegression,0.967000,0.986000,0.967000,0.967000,0.022000
1,Naive Bayes,0.960000,0.960000,0.960000,0.960000,0.019000
2,K nearest neighbour,0.883000,0.908000,0.883000,0.883000,0.030000
3,DecisionTreeClassifier,0.927000,0.932000,0.927000,0.927000,0.014000
4,RandomForestClassifier,0.957000,0.960000,0.957000,0.957000,0.185000
5,RandomForestClassifier,0.956667,0.959732,0.956667,0.956666,0.182906
6,RandomForestClassifier,0.956667,0.959732,0.956667,0.956666,0.221459


In [118]:
conclusion.drop_duplicates(subset= 'Model', inplace= True)
conclusion= conclusion.round(3)
conclusion

,Model,Accuracy,Prediction,Recall,F1 score,Time taken
0,LogisticRegression,0.967,0.986,0.967,0.967,0.022
1,Naive Bayes,0.960,0.960,0.960,0.960,0.019
2,K nearest neighbour,0.883,0.908,0.883,0.883,0.030
3,DecisionTreeClassifier,0.927,0.932,0.927,0.927,0.014
4,RandomForestClassifier,0.957,0.960,0.957,0.957,0.185


**Conclusion:**
* People over 40 are more likely to click on ads.
* People under 40 spend more time on the site but less likely to click on ads.
* Age is the most important factor.

**Next step:**
* Better targeted ads at people less than 40.
* Make site simpler to use to increase usage time of people over 40.